## Extracting Tabular Data From Text

In this example, we will create a basic agent that can extract tabular data from unstructured text using OpenAI function calls.

**Step 1** Let's set up an Azure OpenAI Client and then use ActionWeaver patch to enhance its API


In [1]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

**Step 2** We will use Pydantic Model to define structures we want to extract from the raw text.

In [2]:
# Example inspired by https://jxnl.github.io/instructor/examples/autodataframe/
from typing import Any
from typing import List

from agentkit.llms.client.chat import completion
from pydantic import BaseModel
from pydantic import Field


class Dataframe(BaseModel):
    """
    Class representing a dataframe.
    """
    name: str = Field(..., description="The name of the dataframe")
    data: List[List[Any]] = Field(
        ...,
        description="""Correct rows of data aligned to column names, Nones are allowed.""",
    )
    columns: List[str] = Field(
        ...,
        description="Column names relevant from source data, should be in snake_case",
    )

    def to_pandas(self):
        import pandas as pd
        return pd.DataFrame(data=self.data, columns=self.columns)


    def chat(self, query):
        messages = [
            {
                "role": "system",
                "content": f"You're a helpful agent trying to answer questions based on {str(self)}",
            },
            {
                "role": "user",
                "content": f"{query}",
            },
        ]
        return completion(
            model='gpt-4o',
            messages=messages,
            temperature=1,
            stream=False,
        )

class Database(BaseModel):
    """
    A set of dataframes
    """
    tables: List[Dataframe] = Field(..., description="List of dataframes")

**Step 3**: Utilize the **action_from_model** function to convert the PyDantic model into an ActionWeaver action. Then, employ the `invoke` method to evaluate an example. By using `force=True`, you compel the LLM to execute the action.

In [3]:
from agentkit.actions.factories.pydantic_model_to_action import action_from_model
from agentkit.llms.client.chain import chain_completion

data ="""Mary, 35, from Chicago, is a soccer aficionado. She heads the 'Falcons,' a soccer team with 14 members.

David, a 28-year-old swimming enthusiast from Miami, leads the 'Sharks,' a team of 12 swimmers.

Emily, 22, in Boston, is devoted to volleyball. She captains the 'Eagles,' which includes 16 volleyball players.

Chris, 32, from Seattle, is an avid cyclist and heads the 'Wolves,' a 12-member cycling team.

Anna, 27, in Denver, excels in golf and captains the 'Bears,' a team of 9 golfers.

In Dallas, there's Leo, 24, who loves hockey. He leads the 'Panthers,' a hockey team of 13.

Grace, 29, in Atlanta, is passionate about rowing. She captains the 'Dolphins,' a rowing team of 11 athletes.

In Phoenix, Alex, 26, is a skilled archer and heads the 'Hawks,' an archery team with 8 members.

Finally, in Portland, we have Zoe, 31, who is a master at badminton. She captains the 'Rabbits,' a badminton team of 10 players.
"""

db = chain_completion(
    model='gpt-4o',
    messages=[{"role": "user", "content": data}],
    actions=[action_from_model(Database)],
    stream=False,
)

db

[Database(tables=[Dataframe(name='Team_Leaders', data=[['Mary', 35, 'Chicago', 'Soccer', 'Falcons', 14], ['David', 28, 'Miami', 'Swimming', 'Sharks', 12], ['Emily', 22, 'Boston', 'Volleyball', 'Eagles', 16], ['Chris', 32, 'Seattle', 'Cycling', 'Wolves', 12], ['Anna', 27, 'Denver', 'Golf', 'Bears', 9], ['Leo', 24, 'Dallas', 'Hockey', 'Panthers', 13], ['Grace', 29, 'Atlanta', 'Rowing', 'Dolphins', 11], ['Alex', 26, 'Phoenix', 'Archery', 'Hawks', 8], ['Zoe', 31, 'Portland', 'Badminton', 'Rabbits', 10]], columns=['name', 'age', 'city', 'sport', 'team_name', 'team_size'])])]

In [4]:
db[0].tables[0].to_pandas()

ModuleNotFoundError: No module named 'pandas'

Let's wrap it inside of an agent class. This time, we include an additional actions parameter `orch` when calling the chat completion API. This feature will allow us for more precise control over the specific set of tools available to the LLM during each interaction.

Example:
```python
chain_completion(
    messages = ...
    actions=[a1, a2, a3], # First, LLM respond with either a1, a2 or a3, or text without action
    # Define the orchestration logic for actions:
    orch={
        a1.name: [a2, a3],  # If a1 is invoked, the next response will be either a2, a3 or a text response.
        a2.name: a3,      # If a2 is invoked, the next action will be a3
        a3.name: [a4]     # If a3 is invoked, the next response will be a4 or a text response.
        a4.name: None     # If a4 is invoked, the next response will guarantee to be a text message
    }
)
```

For details please take a look at [here](https://github.com/TengHu/Agentkit?tab=readme-ov-file#orchestration-of-actions-experimental )


----
This agent has the capability to perform two actions:

1. ReadDataAndCreateTable: It can read raw text, extract tabular data, and store it in a database.
2. AnswerQuestion: It can provide answers to questions by using queries and information from specific tables.

In [7]:
from agentkit import action


class DBAgent:
    def __init__(self):
        self.db = Database(tables=[])
        self.messages = [{"role": "system", "content": f"You're a helpful agent. You answer questions in plain English. Tables available: {[table.name for table in self.db.tables]}"}]

    @action("ReadDataAndCreateTable")
    def read(self, data: str):
        """
        This method is used to read unstructured data and create tables.
        Args:
            data (str): The unstructured data to be processed
        """
        print(f"[Reading data]: \n {data} \n")
        messages = [
            {
                "role": "system",
                "content": """Map this data into a dataframe and correctly define the correct columns and rows""",
            },
            {
                "role": "user",
                "content": f"{data}",
            },
        ]
        # Utilize the **action_from_model** function to convert the PyDantic model into an Agentkit action. Then, employ the `invoke` method to evaluate an example.
        # Stop=True, so LLM will return the Database immediately
        db = chain_completion(
            messages=messages,
            model='gpt-4o',
            temperature=0.1,
            stream=False,
            actions=[action_from_model(Database, stop=True)]
        )

        # save the table
        self.db.tables.extend(db[0].tables)
        return f"{[table.name for table in self.db.tables]} have been created"


    @action("AnswerQuestion")
    def answer(self, query: str, tables:List[str]):
        """
        Responds to plain English queries by extracting relevant information from predefined data tables.
        This method is specifically designed to process and answer questions that can be addressed using the data available in the provided tables. The tables are not SQL tables.

        Parameters:
        query (str): A plain English question or query that the user wants to be answered. The method processes this query to understand and extract the necessary information from the available tables.
        tables (List[str]): A list of table names or identifiers. These tables contain the data that will be used to answer the query.

        Usage:
        This method should be invoked when there is a need to answer questions based on the information contained in the specified tables. It is essential that the tables provided have the relevant data to address the query.
        """
        candidates = [table for table in self.db.tables if table.name in tables]
        context = [table.to_pandas().to_json() for table in candidates]

        print (f"[Answering]: {query} using context {context} from tables {tables}")

        messages=[
            {
                "role": "system",
                "content": f"You're a helpful agent trying to answer questions based on context: \n {context}"
            },
            {
                "role": "user",
                "content": f"{query}",
            },
        ]

        response = completion(
                  model='gpt-4o-mini',
                  temperature=.5,
                  messages=messages,
                  stream=False,
              )
        return response



    def chat(self, query):
        self.messages.append({"role": "user", "content": f"{query}"})
        response =  chain_completion(
                              model='gpt-4o',
                              messages=self.messages,
                              temperature=.1,
                              actions = [self.read, self.answer],
                              orch = {self.read.name: None, self.answer.name: None}, # function is called at most once
                              stream=False
                    )
        self.messages.append({"role": "assistant", "content": response.choices[0].message.content})
        return response

agent = DBAgent()

In [8]:
data ="""Mary, 35, from Chicago, is a soccer aficionado. She heads the 'Falcons,' a soccer team with 14 members.

David, a 28-year-old swimming enthusiast from Miami, leads the 'Sharks,' a team of 12 swimmers.

Emily, 22, in Boston, is devoted to volleyball. She captains the 'Eagles,' which includes 16 volleyball players.

Chris, 32, from Seattle, is an avid cyclist and heads the 'Wolves,' a 12-member cycling team.

Anna, 27, in Denver, excels in golf and captains the 'Bears,' a team of 9 golfers.

In Dallas, there's Leo, 24, who loves hockey. He leads the 'Panthers,' a hockey team of 13.

Grace, 29, in Atlanta, is passionate about rowing. She captains the 'Dolphins,' a rowing team of 11 athletes.

In Phoenix, Alex, 26, is a skilled archer and heads the 'Hawks,' an archery team with 8 members.

Finally, in Portland, we have Zoe, 31, who is a master at badminton. She captains the 'Rabbits,' a badminton team of 10 players.
"""

agent.chat(data)

[Reading data]: 
 Mary, 35, from Chicago, is a soccer aficionado. She heads the 'Falcons,' a soccer team with 14 members.

David, a 28-year-old swimming enthusiast from Miami, leads the 'Sharks,' a team of 12 swimmers.

Emily, 22, in Boston, is devoted to volleyball. She captains the 'Eagles,' which includes 16 volleyball players.

Chris, 32, from Seattle, is an avid cyclist and heads the 'Wolves,' a 12-member cycling team.

Anna, 27, in Denver, excels in golf and captains the 'Bears,' a team of 9 golfers.

In Dallas, there's Leo, 24, who loves hockey. He leads the 'Panthers,' a hockey team of 13.

Grace, 29, in Atlanta, is passionate about rowing. She captains the 'Dolphins,' a rowing team of 11 athletes.

In Phoenix, Alex, 26, is a skilled archer and heads the 'Hawks,' an archery team with 8 members.

Finally, in Portland, we have Zoe, 31, who is a master at badminton. She captains the 'Rabbits,' a badminton team of 10 players. 



ModelResponse(id='chatcmpl-9v6QT1fG23obZwZGFmUnyabXU0qkn', choices=[Choices(finish_reason='stop', index=0, message=Message(content='I have organized the information into a table for you:\n\n| Name  | Age | City     | Sport      | Team Name | Team Members |\n|-------|-----|----------|------------|-----------|--------------|\n| Mary  | 35  | Chicago  | Soccer     | Falcons   | 14           |\n| David | 28  | Miami    | Swimming   | Sharks    | 12           |\n| Emily | 22  | Boston   | Volleyball | Eagles    | 16           |\n| Chris | 32  | Seattle  | Cycling    | Wolves    | 12           |\n| Anna  | 27  | Denver   | Golf       | Bears     | 9            |\n| Leo   | 24  | Dallas   | Hockey     | Panthers  | 13           |\n| Grace | 29  | Atlanta  | Rowing     | Dolphins  | 11           |\n| Alex  | 26  | Phoenix  | Archery    | Hawks     | 8            |\n| Zoe   | 31  | Portland | Badminton  | Rabbits   | 10           |\n\nFeel free to ask if you need more details or have any other 

In [9]:
agent.db.tables

[Dataframe(name='team_captains', data=[['Mary', 35, 'Chicago', 'Falcons', 'soccer', 14], ['David', 28, 'Miami', 'Sharks', 'swimming', 12], ['Emily', 22, 'Boston', 'Eagles', 'volleyball', 16], ['Chris', 32, 'Seattle', 'Wolves', 'cycling', 12], ['Anna', 27, 'Denver', 'Bears', 'golf', 9], ['Leo', 24, 'Dallas', 'Panthers', 'hockey', 13], ['Grace', 29, 'Atlanta', 'Dolphins', 'rowing', 11], ['Alex', 26, 'Phoenix', 'Hawks', 'archery', 8], ['Zoe', 31, 'Portland', 'Rabbits', 'badminton', 10]], columns=['name', 'age', 'city', 'team_name', 'sport', 'team_members'])]

In [10]:
agent.chat("find out people below age 30")

ModuleNotFoundError: No module named 'pandas'

In [11]:
agent.chat("""create new table from it""")


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



BadRequestError: litellm.BadRequestError: OpenAIException - Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_Mf5ixjumLnpri0mjX3jMhuQw", 'type': 'invalid_request_error', 'param': 'messages.[7].role', 'code': None}}

In [12]:
len(agent.db.tables)

2